In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import os
import torch
from shutil import copyfile
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

from vocab import Vocab
from dataset import ViOCD_Dataset, collate_fn
from transformer_encoder_model import TransformerEncoderModelLPE,TransformerEncoderModel, TransformerEncoderModelPyTorch

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
scorers = {
    "f1": f1_score,
    "precision": precision_score,
    "recall": recall_score
}

def train(epoch: int, model: nn.Module, dataloader: DataLoader, optim: torch.optim.Optimizer):
    model.train()

    running_loss = .0
    with tqdm(desc='Epoch %d - Training' % epoch, unit='it', total=len(dataloader)) as pbar:
        for it, items in enumerate(dataloader):
            # forward pass
            input_ids = items["input_ids"].to(device)
            labels = items["labels"].to(device)
            
            _, loss = model(input_ids, labels)
            
            # backward pass
            optim.zero_grad()
            loss.backward()
            optim.step()
            running_loss += loss.item()

            # update the training status
            pbar.set_postfix(loss=running_loss / (it + 1))
            pbar.update()

def compute_scores(predictions: list, labels: list) -> dict:
    scores = {}
    for scorer_name in scorers:
        scorer = scorers[scorer_name]
        scores[scorer_name] = scorer(labels, predictions, average="macro")

    return scores

def evaluate_metrics(epoch: int, model: nn.Module, dataloader: DataLoader) -> dict:
    model.eval()
    all_labels = []
    all_predictions = []
    scores = {}
    with tqdm(desc='Epoch %d - Evaluating' % epoch, unit='it', total=len(dataloader)) as pbar:
        for items in dataloader:
            input_ids = items["input_ids"].to(device)
            labels = items["labels"].to(device)
            with torch.no_grad():
                logits, _ = model(input_ids, labels)

            predictions = logits.argmax(dim=-1).long()
    
            labels = labels.view(-1).cpu().numpy()
            predictions = predictions.view(-1).cpu().numpy()

            all_labels.extend(labels)
            all_predictions.extend(predictions)

            pbar.update()
        # Calculate metrics
    scores = compute_scores(all_predictions, all_labels)

    return scores

def save_checkpoint(dict_to_save, checkpoint_path):
    if not os.path.isdir(checkpoint_path):
        os.mkdir(checkpoint_path)
    torch.save(dict_to_save, os.path.join(f"{checkpoint_path}", "last_model.pth"))

def main(
        d_model: int,
        layer_dim: int,
        head: int,
        d_ff: int,
        dropout: float,
        train_path: str, 
        dev_path: str, 
        test_path: str,
        learning_rate,
        checkpoint_path,
        typee):

    vocab = Vocab(
        train_path, dev_path, test_path
    )

    train_dataset = ViOCD_Dataset(train_path, vocab)
    dev_dataset = ViOCD_Dataset(dev_path, vocab)
    test_dataset = ViOCD_Dataset(test_path, vocab)

    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=2,
        collate_fn=collate_fn
    )
    dev_dataloader = DataLoader(
        dataset=dev_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn
    )
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn
    )

    epoch = 0
    score_name = "f1"
    allowed_patience = 5
    best_score = 0

    if typee == 'Câu 3':
        model = TransformerEncoderModelPyTorch(
            d_model, head, layer_dim, d_ff, dropout, vocab
        ).to(device)
    elif typee == 'Câu 1':
        model = TransformerEncoderModel(
            d_model, head, layer_dim, d_ff, dropout, vocab
        ).to(device)
    else:
        model = TransformerEncoderModelLPE(
            d_model, head, layer_dim, d_ff, dropout, vocab
        ).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.98))
    
    while True:
        train(epoch, model, train_dataloader, optim)
        # val scores
        scores = evaluate_metrics(epoch, model, dev_dataloader)
        print(f"Dev scores: {scores}")
        score = scores[score_name]

        # Prepare for next epoch
        is_the_best_model = False
        if score > best_score:
            best_score = score
            patience = 0
            is_the_best_model = True
        else:
            patience += 1

        exit_train = False
        
        if patience == allowed_patience:
            exit_train = True

        save_checkpoint({
            "epoch": epoch,
            "best_score": best_score,
            "patience": patience,
            "state_dict": model.state_dict(),
            "optimizer": optim.state_dict()
        }, checkpoint_path)

        if is_the_best_model:
            copyfile(
                os.path.join(checkpoint_path, "last_model.pth"), 
                os.path.join(checkpoint_path, "best_model.pth")
            )

        if exit_train:
            break

        epoch += 1

    scores = evaluate_metrics(epoch, model, test_dataloader)
    print(f"Test scores: {scores}")

# Câu 1

In [9]:

main(
        d_model=512,
        head=8,
        layer_dim=3,
        d_ff=4086,
        dropout=0.1,
        learning_rate=0.001,
        train_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\train.json",
        dev_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\dev.json",
        test_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\test.json",
        checkpoint_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\checkpoints",
        typee = 'Câu 1'
    )

Epoch 0 - Evaluating: 100%|██████████| 548/548 [00:01<00:00, 277.10it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 1 - Evaluating: 100%|██████████| 548/548 [00:01<00:00, 287.65it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 2 - Evaluating: 100%|██████████| 548/548 [00:02<00:00, 213.34it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 5 - Evaluating: 100%|██████████| 548/548 [00:02<00:00, 234.39it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 5 - Evaluating: 100%|██████████| 549/549 [00:02<00:00, 235.22it/s]

Test scores: {'f1': 0.32967032967032966, 'precision': 0.2459016393442623, 'recall': 0.5}



c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Câu 2

In [10]:
main(
        d_model=512,
        head=8,
        layer_dim=3,
        d_ff=4086,
        dropout=0.1,
        learning_rate=0.001,
        train_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\train.json",
        dev_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\dev.json",
        test_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\test.json",
        checkpoint_path="checkpoints",
        typee = 'Câu 2'
    )

Epoch 0 - Evaluating: 100%|██████████| 548/548 [00:06<00:00, 90.59it/s] 
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 1 - Evaluating: 100%|██████████| 548/548 [00:06<00:00, 89.94it/s] 
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 2 - Evaluating: 100%|██████████| 548/548 [00:08<00:00, 60.90it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 5 - Evaluating: 100%|██████████| 548/548 [00:05<00:00, 108.60it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 5 - Evaluating: 100%|██████████| 549/549 [00:05<00:00, 101.06it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test scores: {'f1': 0.32967032967032966, 'precision': 0.2459016393442623, 'recall': 0.5}


# Câu 3

In [11]:
main(
        d_model=512,
        head=8,
        layer_dim=3,
        d_ff=4086,
        dropout=0.1,
        learning_rate=0.001,
        train_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\train.json",
        dev_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\dev.json",
        test_path=r"C:\Users\VIET HOANG - VTS\Desktop\VisionReader\deeplearning\BTH5\ViOCD\test.json",
        checkpoint_path="checkpoints",
        typee = 'Câu 3'
    )

c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 0 - Training:   0%|          | 0/138 [00:00<?, ?it/s]c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 0 - Evaluating: 100%|██████████| 548/548 [00:02<00:00, 216.88it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\sit

Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 1 - Evaluating: 100%|██████████| 548/548 [00:02<00:00, 205.67it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 2 - Evaluating: 100%|██████████| 548/548 [00:04<00:00, 133.19it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 5 - Evaluating: 100%|██████████| 548/548 [00:01<00:00, 295.73it/s]
c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev scores: {'f1': 0.32595325953259535, 'precision': 0.2417883211678832, 'recall': 0.5}


Epoch 5 - Evaluating: 100%|██████████| 549/549 [00:02<00:00, 236.79it/s]

Test scores: {'f1': 0.32967032967032966, 'precision': 0.2459016393442623, 'recall': 0.5}



c:\Users\VIET HOANG - VTS\Desktop\VisionReader\cuda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
